## Problem Description

A manufacturer needs to refine several raw oils and blend them together to produce a given food product that can be sold. The raw oils needed can be divided into two categories:


| Category        | Oil         |
| ------------- |-------------| 
| Vegetable oils:|  VEG 1<br>VEG 2 | 
| Non-vegetable oils:     | OIL 1<br>OIL 2<br>OIL 3      |


The manufacturer can choose to buy raw oils for the current month and/or buy them on the futures market for delivery in a subsequent month. Prices for immediate delivery and in the futures market are given below in USD/ton:

| Month |	VEG 1 |	VEG 2 |	OIL 1 |	OIL 2 |	OIL 3|
| ------------- |-------------| -------------| -------------| -------------| -------------| 
| 1 = January| 110 |	120 |	130 |	110 |	115|
| 2 = February |130 |	130 |	110 |	90|	115|
| 3 = March |110 |	140 |	130 |	100 |	95|
| 4 = April |120 |	110 |	120 |	120 |	125|
| 5 = May | 100 |	120 |	150 |	110 |	105|
| 6 = June |  	90 |	100 |	140 |	80| 	135 |

There are a number of additional factors that must be taken into account. These include:

1. The final food product sells for $\$150$ per ton.
2. Each category of oil (vegetable and non-vegetable) needs to be refined on a different production line.
3. There is limited refinement capacity such that in any given month a maximum of 200 tons of vegetable oils and 250 tons of non-vegetable oils can be refined.
4. Also, there is no waste in the refinement process, so the sum of the raw oils refined will equal the amount of refined oils available.
5. The cost of refining the oils may be ignored.

In addition to the refining limits above, there are limits to the amount of raw oils that can be stored for future use, and there is a cost associated with each ton of oil stored. The limit is 1,000 tons of each raw oil and the storage cost is $\$5$ per ton per month. The manufacturer cannot store the produced food product or the refined oils.

The final food product must have a hardness between three and six on a given hardness scale. For the purposes of the model, hardness blends linearly and the hardness of each raw oil is:

|Oils |	Hardness|
| ------------- |-------------| 
|VEG 1 |	8.8|
|VEG 2 |	6.1|
|OIL 1 |	2.0|
|OIL2 |	4.2|
|OIL 3| 	5.0|

At the start of January, there are 500 tons of each type of raw oil in storage. For the purpose of the model, this should also be the level of raw oils in storage at the end of June.

This version of the Food Manufacture problem adds the following additional constraints to the first version:

- Condition 1: If an oil is used during a month, the minimum quantity used must be 20 tons.
- Condition 2: The maximum number of oils used in a month is three.
- Condition 3: The use of VEG1 or VEG2 in a given month requires the use of OIL3 in that same month.


Given the above information, what monthly buying and manufacturing decisions should be made in order to maximize profit?

**Reference:**

*This problem is taken from Gurobi’s resources (https://www.gurobi.com/resource/food-manufacturing-i-and-ii/) which is based on an example from a published book (example #2 of the fifth edition of Modeling Building in Mathematical Programming by H. P. Williams on pages 255 and 299 – 300.).*

## Solution

**Sets and Indices:**

$m \in Months = \{1,2,3,4,5,6\}$: Set of months.

$v \in VegOil = \{1,2\}$: Set of vegetable oils.

$n \in NonVegOil = \{1,2,3\}$: Set of non-vegetable oils.

**Parameters:**

$VegCost_{m,v} \in \mathbb{R}^+$: Cost (in USD) of vegetable oil $v$ in month $m$.

$NonVegCost_{m,n} \in \mathbb{R}^+$: Cost (in USD) of non-vegetable oil $n$ in month $m$.

$Product\_Cost = 150$: Price (in USD) of final product.

$Storage\_Cost = 5$: Individual oil storage cost (in USD) per ton per month.

$VegHardness_{v} \in \mathbb{R}^+$: Hardness of vegetable oil $v$.

$NonVegHardness_{n} \in \mathbb{R}^+$: Hardness of non-vegetable oil $n$.

$Init\_Storage = 500$: Initial individual oil storage (in ton) in month $1$.

$Target\_Storage = 500$: Final individual oil storage in (in ton) month $6$.

**Variables:**

$veg\_buy_{m, v} \in \mathbb{R}^+$: Tons of vegetable oil $v$ to buy in month $m$.

$nonveg\_buy_{m, n} \in \mathbb{R}^+$: Tons of non-vegetable oil $n$ to buy in month $m$.

$veg\_consume_{m, v} \in \mathbb{R}^+$: Tons of vegetable oil $v$ to consume in month $m$.

$nonveg\_consume_{m, n} \in \mathbb{R}^+$: Tons of non-vegetable oil $n$ to consume in month $m$.

$veg\_store_{m, v} \in \mathbb{R}^+$: Tons of vegetable oil $v$ to store in month $m$.

$nonveg\_store_{m, n} \in \mathbb{R}^+$: Tons of non-vegetable oil $n$ to store in month $m$.

$veg\_use_{m, v} \in \{0,1\}$: 1 if vegetable oil $v$ is used in month $m$, else 0.

$nonveg\_use_{m, n} \in \{0,1\}$: 1 if non-vegetable oil $n$ is used in month $m$, else 0.

$produce_{m} \in \mathbb{R}^+$: Total food product produced (in tons) in month $m$.

**Objective Function:**

\begin{equation}
\text{Max} \quad Z = \sum_{m \in Months} Product\_Cost*produce_{m} - \sum_{m \in Months} \left(\sum_{v \in VegOil} VegCost_{m,v}*veg\_buy_{m,v} + Storage\_Cost*veg\_store_{m,v} + \\ \sum_{n \in NonVegOil} NonVegCost_{m,n}*nonveg\_buy_{m,n} + Storage\_Cost*nonveg\_store_{m,n}\right)
\tag{0}
\end{equation}

**Constraints:**

\begin{equation}
Init\_Storage + veg\_buy_{1,v} = veg\_consume_{1,v} + veg\_store_{1,v} \quad \forall v \in VegOil
\tag{1a}
\end{equation}

\begin{equation}
Init\_Storage + nonveg\_buy_{1,n} = nonveg\_consume_{1,n} + nonveg\_store_{1,n} \quad \forall n \in NonVegOil
\tag{1b}
\end{equation}

\begin{equation}
veg\_store_{m-1, v} + veg\_buy_{m,v} = veg\_consume_{m,v} + veg\_store_{m,v} \quad \forall (m,v) \in Months \setminus \{1\} \times VegOil
\tag{2a}
\end{equation}

\begin{equation}
nonveg\_store_{m-1,n} + nonveg\_buy_{m,n} = nonveg\_consume_{m,n} + nonveg\_store_{m,n} \quad \forall (m,n) \in Months \setminus \{1\} \times NonVegOil
\tag{2b}
\end{equation}

\begin{equation}
veg\_store_{6,v} = Target\_Storage \quad \forall v \in VegOil
\tag{3a}
\end{equation}

\begin{equation}
nonveg\_store_{6,n} = Target\_Storage \quad \forall n \in NonVegOil
\tag{3b}
\end{equation}

\begin{equation}
\sum_{v \in VegOil} veg\_consume_{m,v} \le 200 \quad \forall m \in Months
\tag{4a}
\end{equation}

\begin{equation}
\sum_{n \in NonVegOil} nonveg\_consume_{m,n} \le 250 \quad \forall m \in Months
\tag{4b}
\end{equation}

\begin{equation}
3*produce_{m} \le \sum_{v \in VegOil} VegHardness_{v}*veg\_consume_{m,v} + \\
\sum_{n \in NonVegOil} NonVegHardness_{n}*nonveg\_consume_{m,n} \le 6*produce_{m} \quad \forall m \in Months
\tag{5}
\end{equation}

\begin{equation}
\sum_{v \in VegOil} veg\_consume_{m,v} + \sum_{n \in NonVegOil} nonveg\_consume_{m,n} = produce_{m} \quad \forall m \in Months
\tag{6}
\end{equation}

\begin{equation}
20*veg\_use_{m,v} \le veg\_consume_{m,v} \le 200*veg\_use_{m,v} \quad \forall (m,v) \in Months \times VegOil
\tag{7a}
\end{equation}

\begin{equation}
20*nonveg\_use_{m,n} \le nonveg\_consume_{m,n} \le 250*nonveg\_use_{m,n} \quad \forall (m,n) \in Months \times NonVegOil
\tag{7b}
\end{equation}

\begin{equation}
\sum_{v \in VegOil} veg\_use_{m,v} + \sum_{n \in NonVegOil} nonveg\_use_{m,n} \le 3 \quad \forall m \in Months
\tag{8}
\end{equation}

\begin{equation}
veg\_use_{m,1} + veg\_use_{m,2} \le 2*nonveg\_use_{m,3} \quad \forall m \in Months
\tag{9}
\end{equation}

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyomo
import pandas as pd

     |████████████████████████████████| 9.2 MB 5.6 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [ ]:
model = pyomo.ConcreteModel()

# Sets
model.months = pyomo.RangeSet(6)
model.veg = pyomo.RangeSet(2)
model.nonveg = pyomo.RangeSet(3)

# Variables
model.veg_buy = pyomo.Var(model.months, model.veg, domain = pyomo.NonNegativeReals)
model.nonveg_buy = pyomo.Var(model.months, model.nonveg, domain = pyomo.NonNegativeReals)

model.veg_consume = pyomo.Var(model.months, model.veg, domain = pyomo.NonNegativeReals)
model.nonveg_consume = pyomo.Var(model.months, model.nonveg, domain = pyomo.NonNegativeReals)

model.veg_store = pyomo.Var(model.months, model.veg, domain = pyomo.NonNegativeReals)
model.nonveg_store = pyomo.Var(model.months, model.nonveg, domain = pyomo.NonNegativeReals)

model.veg_use = pyomo.Var(model.months, model.veg, domain = pyomo.Binary)
model.nonveg_use = pyomo.Var(model.months, model.nonveg, domain = pyomo.Binary)

model.produce = pyomo.Var(model.months, domain = pyomo.NonNegativeReals)

# Parameters
model.veg_cost = pyomo.Param(model.months, model.veg, initialize = {(1,1):110, (2,1):130, (3,1):110, (4,1):120, (5,1):100, (6,1):90,
                                                                    (1,2):120, (2,2):130, (3,2):140, (4,2):110, (5,2):120, (6,2):100})

model.nonveg_cost = pyomo.Param(model.months, model.nonveg, initialize = {(1,1):130, (2,1):110, (3,1):130, (4,1):120, (5,1):150, (6,1):140,
                                                                          (1,2):110, (2,2):90, (3,2):100, (4,2):120, (5,2):110, (6,2):80,
                                                                          (1,3):115, (2,3):115, (3,3):95, (4,3):125, (5,3):105, (6,3):135})

model.veg_hardness = pyomo.Param(model.veg, initialize = {1:8.8, 2:6.1})

model.nonveg_hardness = pyomo.Param(model.nonveg, initialize = {1:2.0, 2:4.2, 3:5.0})

model.product_cost = pyomo.Param(initialize = 150)
model.storage_cost = pyomo.Param(initialize = 5)
model.init_storage = pyomo.Param(initialize = 500)
model.target_storage = pyomo.Param(initialize = 500)

# Objective Function
def obj(model):
  value = sum(model.product_cost*model.produce[m] for m in model.months) - \
          sum(sum(model.veg_cost[m,v]*model.veg_buy[m,v] + model.storage_cost*model.veg_store[m,v] for v in model.veg) for m in model.months) + \
          sum(sum(model.nonveg_cost[m,n]*model.nonveg_buy[m,n] + model.storage_cost*model.nonveg_store[m,n] for n in model.nonveg) for m in model.months)

  return value
model.obj = pyomo.Objective(rule = obj, sense = pyomo.maximize)

# Constraints
def rule1a(model, m, v):
  if m == 1:
    return model.init_storage + model.veg_buy[m,v] == model.veg_consume[m,v] + model.veg_store[m,v]
  if m != 1:
    return model.veg_store[m-1,v] + model.veg_buy[m,v] == model.veg_consume[m,v] + model.veg_store[m,v]
model.rule1a = pyomo.Constraint(model.months, model.veg, rule = rule1a)

def rule1b(model, m, n):
  if m == 1:
    return model.init_storage + model.nonveg_buy[m,n] == model.nonveg_consume[m,n] + model.nonveg_store[m,n]
  if m != 1:
    return model.nonveg_store[m-1,n] + model.nonveg_buy[m,n] == model.nonveg_consume[m,n] + model.nonveg_store[m,n]
model.rule1b = pyomo.Constraint(model.months, model.nonveg, rule = rule1b)

def rule2a(model, v):
  return model.veg_store[6,v] == model.target_storage
model.rule2a = pyomo.Constraint(model.veg, rule = rule2a)

def rule2b(model, n):
  return model.nonveg_store[6,n] == model.target_storage
model.rule2b = pyomo.Constraint(model.nonveg, rule = rule2b)

def rule3a(model, m):
  return sum(model.veg_consume[m,v] for v in model.veg) <= 200
model.rule3a = pyomo.Constraint(model.months, rule = rule3a)

def rule3b(model, m):
  return sum(model.nonveg_consume[m,n] for n in model.nonveg) <= 250
model.rule3b = pyomo.Constraint(model.months, rule = rule3b)

def rule4a(model, m):
  return 3*model.produce[m] <= sum(model.veg_hardness[v]*model.veg_consume[m,v] for v in model.veg) + \
                                sum(model.nonveg_hardness[n]*model.nonveg_consume[m,n] for n in model.nonveg)
model.rule4a = pyomo.Constraint(model.months, rule = rule4a)

def rule4b(model, m):
  return sum(model.veg_hardness[v]*model.veg_consume[m,v] for v in model.veg) + sum(model.nonveg_hardness[n]*model.nonveg_consume[m,n] for n in model.nonveg) \
          <= 6*model.produce[m]
model.rule4b = pyomo.Constraint(model.months, rule = rule4b)

def rule5(model, m):
  return sum(model.veg_consume[m,v] for v in model.veg) + sum(model.nonveg_consume[m,n] for n in model.nonveg) == model.produce[m]
model.rule5 = pyomo.Constraint(model.months, rule = rule5)

def rule6a(model, m, v):
  return model.veg_consume[m,v] <= 200*model.veg_use[m,v]
model.rule6a = pyomo.Constraint(model.months, model.veg, rule = rule6a, doc = 'Part 1 for the veg oil either-or constraint')

def rule6b(model, m, v):
  return 20 - model.veg_consume[m,v] <=  200*(1-model.veg_use[m,v])
model.rule6b = pyomo.Constraint(model.months, model.veg, rule = rule6b, doc = 'Part 2 for the veg oil either-or constraint')

def rule7a(model, m, n):
  return model.nonveg_consume[m,n] <= 250*model.nonveg_use[m,n]
model.rule7a = pyomo.Constraint(model.months, model.nonveg, rule = rule7a, doc = 'Part 1 for the nonveg oil either-or constraint')

def rule7b(model, m, n):
  return 20 - model.nonveg_consume[m,n] <=  250*(1-model.nonveg_use[m,n])
model.rule7b = pyomo.Constraint(model.months, model.nonveg, rule = rule7b, doc = 'Part 2 for the nonveg oil either-or constraint')

def rule8(model, m):
  return sum(model.veg_use[m,v] for v in model.veg) + sum(model.nonveg_use[m,n] for n in model.nonveg) <= 3
model.rule8 = pyomo.Constraint(model.months, rule = rule8)

def rule9(model, m):
  return model.veg_use[m,1] + model.veg_use[m,2] <= 2*model.nonveg_use[m,3]
model.rule9 = pyomo.Constraint(model.months, rule = rule9)

# Solver options
results = pyomo.SolverFactory('glpk',executable='/usr/bin/glpsol').solve(model)

# Printing results
results.write()

print('\n RESULTS \n')
print('The maximum profit that can be obtained is ',model.obj(),'$')

def prepare_df(veg_data, nonveg_data):
  df_veg = pd.DataFrame(columns = model.veg, index = model.months)
  for m in model.months:
    for v in model.veg:
      df_veg[v][m] = veg_data[m,v]()
  df_nonveg = pd.DataFrame(columns = model.nonveg, index = model.months)
  for m in model.months:
    for v in model.nonveg:
      df_nonveg[v][m] = nonveg_data[m,v]()
  df_fin = pd.concat([df_veg,df_nonveg], axis = 1)
  df_fin.columns = ['Veg1','Veg2','Oil1','Oil2','Oil3']
  df_fin.index = ['Jan','Feb','Mar','Apr','May','Jun']
  return df_fin

df_buy = prepare_df(model.veg_buy, model.nonveg_buy)
df_consume = prepare_df(model.veg_consume, model.nonveg_consume)
df_store = prepare_df(model.veg_store, model.nonveg_store)
df_use = prepare_df(model.veg_use, model.nonveg_use)

print('\n The Consume Table \n')
print(df_consume)
print('\n The Buy Table \n')
print(df_buy)
print('\n The Store Table \n')
print(df_store)
print('\n The Use Table \n')
print(df_use)

df_produce = pd.DataFrame(columns = model.months)
for m in model.months:
  df_produce.loc[0,m] = model.produce[m]()
df_produce.columns = ['Jan','Feb','Mar','Apr','May','Jun']
print('\n Monthly Produce \n')
print(df_produce)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 533450.0
  Upper bound: 533450.0
  Number of objectives: 1
  Number of constraints: 138
  Number of variables: 127
  Number of nonzeros: 427
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 45
      Number of created subproblems: 45
  Error rc: 0
  Time: 0.019038677215576172
# ----------------------------------------------------------
#   Solution Information
# ----------------------